In [4]:
from abc import ABC, abstractmethod
import numpy as np
import pickle

class InterfaceEnvironment(ABC):
    
    @abstractmethod
    def __init__(self):
        pass
    
    @property
    @abstractmethod
    def states(self):
        """The representation of the state space"""
        pass
    
    
    @abstractmethod
    def step(self, action):
        """Takes a step. Uses action for the step
        
        returns
        =======
        
        
        """
        pass
    
    @abstractmethod
    def reset(self):
        """resets environment. Defaults back to starting period. 
        reset method also can take a state (used for solving the model)"""
        pass
    
    

In [72]:
with open('..//data//fertility.pkl', 'rb' ) as f:
    fertility_data = pickle.load(f)
    
with open('..//data//men_wage_path.pkl', 'rb' ) as f:
    men_wage_path_data = pickle.load(f)
    
with open('..//data//men_hours_empirical', 'rb') as f:
    men_hours_data = pickle.load(f)
    
men_salary_path = np.array(men_hours_data * men_wage_path_data * 46)

In [73]:
# calculating scales
Q_mean, Q_scale = (60 + 18)*0.5, (60 - 18)*0.5
K_mean, K_scale = (0 + 5)*0.5, (5 - 0)*0.5
G_mean, G_scale = (0 + 5)*0.5, (5 - 0)*0.5
Z_mean, Z_scale = (-200 + 200), (200 - (-200))*0.5

beta_K_mean, beta_K_scale =(-5 + 5)*0.5, (5 - (-5))*0.5
beta_L_mean, beta_L_scale =(-5 + 5)*0.5, (5 - (-5))*0.5

In [74]:
(-200 - Z_mean )/ Z_scale

-1.0

In [75]:
def scale_states(Q, G, K, Z, beta_K, beta_L):
    Q = (Q - Q_mean )/ Q_scale
    G = (G - Q_mean )/ G_scale
    K = (K - K_mean )/ K_scale
    Z = (Z - Z_mean )/ Z_scale
    
    beta_K = (beta_K - beta_K_mean )/ beta_K_scale
    beta_L = (beta_L - beta_L_mean )/ beta_L_scale
    return np.array([[Q, G, K, Z, beta_K, beta_L]])
    
class EnvironmentModel1(InterfaceEnvironment):
    
    """
    Ordering of items
    states: Q, M, K, W
    shocks: epsilon, rho, psi
    """
    
    DEFAULT_Q = 18
    DEFAULT_G = 2.0
    DEFAULT_K = 0
    DEFAULT_Z = 0.0
    
    def __init__(self, sigma_epsilon, S_min, eta_G, eta_G_sq, alpha, delta, beta_K, beta_L):
        
        #parameters
        self.sigma_epsilon = sigma_epsilon
        self.S_min = S_min
        self.eta_G = eta_G
        self.eta_G_sq = eta_G_sq
        self.alpha = alpha
        self.delta = delta
        
        # The parameters that need to be tuned!
        self.beta_K = beta_K
        self.beta_L = beta_L

        #states
        self.Q = self.DEFAULT_Q
        self.G = self.DEFAULT_G
        self.K = self.DEFAULT_K
        self.Z = self.DEFAULT_Z
        
    def __repr__(self):
        return f"(Q: {self.Q}, G: {self.G}, K: {self.K}, Z: {self.Z})"
    
    @property
    def states(self):
        return scale_states(self.Q, self.G, self.K, self.Z, self.beta_K, self.beta_L)
    

    
    def reset(self, states=None, parameters=None):
        """Expect states given as: (Q, G, K, Z) """
        if states is not None:
            Q, G, K, Z = states[0], states[1], states[2], states[3]
            self.Q = Q
            self.G = G
            self.K = K
            self.Z = Z
        else:
            self.Q = self.DEFAULT_Q
            self.G = self.DEFAULT_G
            self.K = self.DEFAULT_K
            self.Z = self.DEFAULT_Z
            
        if parameters is not None:
            for key, val in parameters.items():
                setattr(self, key, val)
        
    def step(self, action, shocks=None, parameters=None):
        """
        shocks:
            (epsilon, psi) <- that order
        """
        if shocks is None:
             shocks = self.draw_shocks()
        epsilon, psi = shocks

        if parameters is not None:
            for key, val in parameters.items():
                setattr(self, key, val)
        # remember action: hours (H)
    
        ### transition
        self.calc_Q()
        self.calc_G(action)
        self.calc_K(psi)
        self.calc_Z(epsilon)
        
        ### model dynamic
        L = self.calc_L(action)
        
        # wage/salary process
        log_S_tilde = self.calc_log_S_tilde()
        S = self.calc_S(log_S_tilde)
        W = self.calc_W(S, action)

        # husband income
        M = self.calc_M()
        
        # household income
        Y = self.calc_Y(W, M)
        
        utility = self.calc_U(L, Y)
        

        # this might be changed
        done = self.calc_stops()
        
        _info = f'Y: {Y}, L: {L}, W: {W}' 
        if done is True:
            return self.states, utility, True, _info 
        
        return self.states, utility, False, _info

    
    #model dynamic
    def calc_log_S_tilde(self):
        return self.alpha + self.eta_G * self.G + self.eta_G_sq * self.G**2
    
    def calc_U(self, L, Y):
        u = self.beta_K * np.log(self.K + 1) + self.beta_L * np.log(L + 1) + np.log(Y + 1)
        if np.isnan(u):
            raise Exception(f"K: {self.K}, L: {L}, Y: {Y}")
        return u
    
    def calc_W(self, S, H):
        return S * H
    
    def calc_M(self):
        # use data (non parametric)
        return men_salary_path[self.Q]
    
    def calc_Y(self, W, M ):
        return W + M
    
    def calc_L(self, hours):
        return 46 * (7 * 24 - hours)
    
    def calc_stops(self):
        # stops the model (returns done flag)
        if self.Q > 60:
            return True
        return False
    
    def calc_K(self, psi):
        if self.K < 5:
            self.K = self.K + psi
    
    def calc_S(self, log_S_tilde):
        _S = np.exp(log_S_tilde)+ self.Z
        return max(self.S_min, _S)
                           
    def calc_Q(self):
        self.Q = self.Q + 1
    
    def calc_G(self, H):
        self.G = self.G * (1 - self.delta) + H/37
        
    def calc_Z(self, epsilon):
        self.Z = self.Z + epsilon
    
    # def shocks
    def draw_shocks(self):
        return (self.draw_epsilon(), self.draw_psi())
        
    def draw_epsilon(self):
        return np.random.normal(0, self.sigma_epsilon)
    
    def get_p_psi(self):
        return fertility_data[self.Q]
    
    def draw_psi(self):
        p_psi = self.get_p_psi()
        return np.random.binomial(1, p_psi)

    # helpers
    @property
    def observation_space(self):
        return self.states
    
    @property
    def action_space(self):
        return 4

In [76]:
from itertools import product
def create_state_grid():
    Q, K, G, Z, = reversed(list(range(18, 61))), list(range(0,6)), np.linspace(0, 6, num=13), np.linspace(-200, 200, 17)
    return list(product(Q, G, K, Z))
    

In [77]:
parameters = {
    'beta_K' : 1,
    'beta_L' : 1,
    'sigma_epsilon' : 0.1, 
    'S_min': 120.0,
    'alpha': 4.609,
    'eta_G': 0.164,
    'eta_G_sq' : 0.015,
    'delta': 0.209,
    'sigma_epsilon': 15.11
}
env = EnvironmentModel1(**parameters)

In [78]:
env.states.shape

(1, 6)

In [87]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

EPISODES = 1000

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        #with tf.device('/gpu:0'):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


        
parameters = {
    'beta_K' : 1,
    'beta_L' : 1,
    'sigma_epsilon' : 0.1, 
    'S_min': 120.0,
    'alpha': 4.609,
    'eta_G': 0.164,
    'eta_G_sq' : 0.015,
    'delta': 0.209,
    'sigma_epsilon': 15.11
}
env = EnvironmentModel1(**parameters)

state_size = env.observation_space.shape[1]
action_size = 4
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    env.reset()
    state = env.states
    all_rewards = list()
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        all_rewards.append( reward )
        agent.memorize(state, action, reward/30, next_state, done)
        state = next_state
        if done:
            print(reward)
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, np.mean(all_rewards), agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            # Logging training loss every 10 timesteps
            if time % 10 == 0:
                loss = agent.replay(batch_size)
                print("episode: {}/{}, time: {}, loss: {:.4f}"
                    .format(e, EPISODES, time, loss))  

episode: 0/1000, time: 40, loss: 4.9433
23.360694946465927
episode: 0/1000, score: 22.887548367952327, e: 0.99
episode: 1/1000, time: 0, loss: 5.2687
episode: 1/1000, time: 10, loss: 4.6560
episode: 1/1000, time: 20, loss: 4.6369
episode: 1/1000, time: 30, loss: 4.9446
episode: 1/1000, time: 40, loss: 4.2252
23.36647574638525
episode: 1/1000, score: 22.93322451759267, e: 0.97
episode: 2/1000, time: 0, loss: 4.1600
episode: 2/1000, time: 10, loss: 4.1728
episode: 2/1000, time: 20, loss: 4.7803
episode: 2/1000, time: 30, loss: 4.4423
episode: 2/1000, time: 40, loss: 3.4729
21.991634371564366
episode: 2/1000, score: 21.853887570918605, e: 0.95
episode: 3/1000, time: 0, loss: 4.4762
episode: 3/1000, time: 10, loss: 4.7959
episode: 3/1000, time: 20, loss: 3.6833
episode: 3/1000, time: 30, loss: 4.0127
episode: 3/1000, time: 40, loss: 3.8792
23.36647574638525
episode: 3/1000, score: 22.917477118271357, e: 0.92
episode: 4/1000, time: 0, loss: 3.8645
episode: 4/1000, time: 10, loss: 3.8762
epi

KeyboardInterrupt: 

In [80]:
env.states

array([[ -0.95238095, -14.9672    ,  -1.        ,  -0.02798838,
          0.2       ,   0.2       ]])